<a href="https://colab.research.google.com/github/VGGatGitHub/Summer2021/blob/main/solver_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook experimenting how to use DOcplex and IBM Quantum.

# 0. Install libraries & clone repo

In [ ]:
import sys

if hasattr(sys, 'real_prefix'):
    #we are in a virtual env.
    !pip install cplex && pip install docplex
else:
    !pip install --user cplex && pip install --user docplex

print()
!pip show cplex
print()
!pip show docplex

#it my be useful to retstart you runtime

In [3]:
!pip show qiskit
#!pip install qiskit

# !pip install qiskit-optimization
#!pip show qiskit-optimization

#it my be useful to retstart you runtime

Name: qiskit
Version: 0.29.0
Summary: Software for developing quantum computing programs
Home-page: https://github.com/Qiskit/qiskit
Author: Qiskit Development Team
Author-email: hello@qiskit.org
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: qiskit-aqua, qiskit-terra, qiskit-ignis, qiskit-ibmq-provider, qiskit-aer
Required-by: 


Possibly need to restart runtime here.

In [1]:
#it my be useful to retstart you runtime

# clone repo
!git clone https://github.com/VGGatGitHub/Summer2021.git

Cloning into 'Summer2021'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 74 (delta 27), reused 21 (delta 10), pack-reused 0
Unpacking objects: 100% (74/74), done.


# 1. Load Data

In [1]:
#next run the model.py in the folder introScenario
!pwd
%cd ./Summer2021/introScenario/
#%ls


/content
/content/Summer2021/introScenario


In [2]:
import pandas as pd

# supply + costs
supply_file = './plants.csv'

# demand
demand_file = './customerDemand.csv'

# for now load example data (plants.csv and customerDemand.csv under introScenario folder)
s_df = pd.read_csv(supply_file)
d_df = pd.read_csv(demand_file)

In [3]:
inputs={'customerDemand':d_df,'plants':s_df}

# 2. Solve the DO problem via DOcplex

In [4]:
#If you get an error you may have to restart the runtime
import cplex
import docplex.mp
from docplex.mp.model import Model

In [5]:
global output_lock
#with output_lock:

try:  
  if output_lock:
    print(output_lock)
except:
  output_lock=dict({'output_lock':{}})

try: 
  if outputs:
    print(outputs)
except:
  outputs=dict()


In [6]:
exec(open("./model.py").read())

* building wado model
model building done in 0.07274937629699707 secs
* running wado model
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               120
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 20 rows and 12 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.01 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.01 ticks)
Solve status: JobSolveStatus.OPTIMAL_SOLUTION
* model docplex_model1 solved with objective = -105680.000
*  KPI: total plants allocations                  = 112640.000
*  KPI: total Cost of plants over all all

In [7]:
outputs


{'plantsAllocation':    plants  ...  plants minValueAllocationForAssignment
 0       1  ...                                       0
 1       2  ...                                       0
 2       3  ...                                       0
 3       4  ...                                       0
 4       5  ...                                       0
 5       6  ...                                       0
 
 [6 rows x 7 columns]}

In [8]:
outputs['plantsAllocation']

,plants,plants allocation decision,plants selection decision,plants Capacity,plants Product,plants Cost,plants minValueAllocationForAssignment
0,1,0.0,1.0,40,mask,3,0
1,2,15.0,1.0,30,mask,2,0
2,3,30.0,1.0,30,handSanitizer,1,0
3,4,70.0,1.0,100,handSanitizer,3,0
4,5,60.0,1.0,60,mask,2,0
5,6,45.0,1.0,45,mask,1,0


In [9]:
model

docplex.mp.Model['docplex_model1']

In [10]:
solution=model.solve()
print(solution)
solution

solution for: docplex_model1
objective: -105680
x2=15
x3=30
x4=70
x5=60
x6=45
x7=1
x8=1
x9=1
x10=1
x11=1
x12=1



docplex.mp.solution.SolveSolution(obj=-105680,values={x2:15,x3:30,x4:70,..

In [24]:
cost = solution.get_objective_value()
x = solution.get_all_values()
x = np.array(x)
print(cost,x)

-105680.0 [ 0. 15. 30. 70. 60. 45.  1.  1.  1.  1.  1.  1.]


#Minimum Eigen Optimizer example

In [ ]:
#https://quantum-computing.ibm.com/lab/docs/iql/optimization

from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

aqua_globals.random_seed = 10598
quantum_instance = QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                   seed_simulator=aqua_globals.random_seed,
                                   seed_transpiler=aqua_globals.random_seed)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])
exact_mes = NumPyMinimumEigensolver()

qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver
rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=1, min_num_vars_optimizer=exact)


In [26]:
# create a QUBO
qubo = QuadraticProgram()
qubo.binary_var('x')
qubo.binary_var('y')
qubo.binary_var('z')
qubo.minimize(linear=[1,-2,3], quadratic={('x', 'y'): 1, ('x', 'z'): -1, ('y', 'z'): 2})
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



In [27]:
%time exact_result = exact.solve(qubo)
print(exact_result)

CPU times: user 21 ms, sys: 2.17 ms, total: 23.2 ms
Wall time: 23 ms
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


/usr/local/lib/python3.7/dist-packages/qiskit/aqua/operators/operator_base.py:46: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.operators', 'qiskit.opflow', 'qiskit-terra')


In [28]:
op, offset = qubo.to_ising()
qp=QuadraticProgram()
qp.from_ising(op, offset, linear=True)

In [29]:
%time exact_result = exact.solve(qp)
print(exact_result)

CPU times: user 23.2 ms, sys: 6.48 ms, total: 29.7 ms
Wall time: 29.5 ms
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


In [30]:
%time qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/components/variational_forms/variational_form.py:48: DeprecationWarning: The package qiskit.aqua.components.variational_forms is deprecated. For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.components.variational_forms')


CPU times: user 352 ms, sys: 41.1 ms, total: 393 ms
Wall time: 403 ms
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


In [31]:
%time rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)

CPU times: user 230 ms, sys: 3 µs, total: 230 ms
Wall time: 237 ms
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


#Try Grover Optimizer

In [32]:
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram
from qiskit import BasicAer
from docplex.mp.model import Model

backend = BasicAer.get_backend('statevector_simulator')
grover_optimizer = GroverOptimizer(6, num_iterations=10, quantum_instance=backend)


In [33]:
%time results = grover_optimizer.solve(qp)#qubo or qp
print("x={}".format(results.x))
print("fval={}".format(results.fval))

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/algorithms/amplitude_amplifiers/grover.py:216: DeprecationWarning: The package qiskit.aqua.algorithms.amplitude_amplifiers is deprecated. It was moved/refactored to qiskit.algorithms.amplitude_amplifiers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  'qiskit.algorithms.amplitude_amplifiers', 'qiskit-terra')


CPU times: user 50.2 s, sys: 2.17 s, total: 52.4 s
Wall time: 48.4 s
x=[0. 1. 0.]
fval=-2.0


In [34]:
%time results = grover_optimizer.solve(qubo)#qubo or qp
print("x={}".format(results.x))
print("fval={}".format(results.fval))

CPU times: user 55.4 s, sys: 2.14 s, total: 57.6 s
Wall time: 53.5 s
x=[0. 1. 0.]
fval=-2.0


#Try ADMM Optimizer


In [35]:
import time
from typing import List, Optional, Any
import numpy as np
import matplotlib.pyplot as plt 

from docplex.mp.model import Model

from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import CobylaOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram
from qiskit.optimization.algorithms.admm_optimizer import ADMMParameters, ADMMOptimizer

# If CPLEX is installed, you can uncomment this line to import the CplexOptimizer.
# CPLEX can be used in this tutorial to solve the convex continuous problem, 
# but also as a reference to solve the QUBO, or even the full problem.
# 
# from qiskit.optimization.algorithms import CplexOptimizer

In [36]:
# define COBYLA optimizer to handle convex continuous problems.
cobyla = CobylaOptimizer()

# define QAOA via the minimum eigen optimizer
qaoa = MinimumEigenOptimizer(QAOA(quantum_instance=BasicAer.get_backend('statevector_simulator')))

# exact QUBO solver as classical benchmark
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver()) # to solve QUBOs

# in case CPLEX is installed it can also be used for the convex problems, the QUBO, 
# or as a benchmark for the full problem.
#
# cplex = CplexOptimizer()

In [37]:
admm_params = ADMMParameters(
                            rho_initial=1001, 
                            beta=1000, 
                            factor_c=900,
                            maxiter=100,
                            three_block=True, tol=1.e-6
                        )

In [38]:
# define QUBO optimizer
qubo_optimizer = exact
# qubo_optimizer = cplex  # uncomment to use CPLEX instead

# define classical optimizer
convex_optimizer = cobyla
# convex_optimizer = cplex  # uncomment to use CPLEX instead

# initialize ADMM with classical QUBO and convex optimizer
admm = ADMMOptimizer(params=admm_params, 
                     qubo_optimizer=qubo_optimizer,
                     continuous_optimizer=convex_optimizer)

In [39]:
print(qubo)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



In [40]:
# run ADMM to solve problem
%time result = admm.solve(qubo)
print("x={}".format(result.x))
print("fval={:.2f}".format(result.fval))

CPU times: user 11.1 s, sys: 2.94 s, total: 14 s
Wall time: 16.2 s
x=[0. 0. 0.]
fval=0.00


In [41]:
%time result = admm.solve(qp)
print("x={}".format(result.x))
print("fval={:.2f}".format(result.fval))

CPU times: user 11.1 s, sys: 2.89 s, total: 14 s
Wall time: 16.2 s
x=[0. 0. 0.]
fval=0.00


In [42]:
# define QUBO optimizer
qubo_optimizer = qaoa 

# define classical optimizer
convex_optimizer = cobyla
# convex_optimizer = cplex  # uncomment to use CPLEX instead

# initialize ADMM with quantum QUBO optimizer and classical convex optimizer
admm_q = ADMMOptimizer(params=admm_params, 
                       qubo_optimizer=qubo_optimizer,
                       continuous_optimizer=convex_optimizer)

In [43]:
# run ADMM to solve problem
%time  result_q = admm_q.solve(qp) #note the need for 402 qubits for mod

CPU times: user 47.5 s, sys: 3.75 s, total: 51.3 s
Wall time: 52.8 s


# 3. Solve the problem using QISKit

In [44]:
# take a look at https://quantum-computing.ibm.com/lab/docs/iql/optimization

#from qiskit_optimization import QuadraticProgram
#from qiskit_optimization.translators import from_docplex_mp

from qiskit.optimization import QuadraticProgram

# load from a Docplex model
mod = QuadraticProgram()
mod.from_docplex(model)
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - 464 x0 - 480 x1 - 496 x2 - 464 x3 - 480 x4 - 496 x5
Subject To
 _L_EXPR_1: x0 - 40 x6 <= 0
 _L_EXPR_2: x1 - 30 x7 <= 0
 _L_EXPR_3: x2 - 30 x8 <= 0
 _L_EXPR_4: x3 - 100 x9 <= 0
 _L_EXPR_5: x4 - 60 x10 <= 0
 _L_EXPR_6: x5 - 45 x11 <= 0
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_13: x0 <= 40
 _L_EXPR_14: x1 <= 30
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_17: x4 <= 60
 _L_EXPR_18: x5 <= 45
 _L_EXPR_19: x2 + x3 <= 100
 _L_EXPR_20: x0 + x1 + x4 + x5 <= 120

Bounds
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1

Binaries
 x6 x7 x8 x9 x10 x11

Generals
 x0 x1 x2 x3 x4 x5
End



In [45]:
print('constant:\t\t\t', mod.objective.constant)

print('linear dict:\t\t\t', mod.objective.linear.to_dict())
print('linear array:\t\t\t', mod.objective.linear.to_array())
print('linear array as sparse matrix:\n', mod.objective.linear.coefficients, '\n')

print('quadratic dict w/ index:\t', mod.objective.quadratic.to_dict())
print('quadratic dict w/ name:\t\t', mod.objective.quadratic.to_dict(use_name=True))
print('symmetric quadratic dict w/ name:\t', mod.objective.quadratic.to_dict(use_name=True, symmetric=True))
print('quadratic matrix:\n', mod.objective.quadratic.to_array(),'\n')
print('symmetric quadratic matrix:\n', mod.objective.quadratic.to_array(symmetric=True),'\n')
print('quadratic matrix as sparse matrix:\n', mod.objective.quadratic.coefficients)


constant:			 0.0
linear dict:			 {0: -464.0, 1: -480.0, 2: -496.0, 3: -464.0, 4: -480.0, 5: -496.0}
linear array:			 [-464. -480. -496. -464. -480. -496.    0.    0.    0.    0.    0.    0.]
linear array as sparse matrix:
   (0, 0)	-464.0
  (0, 1)	-480.0
  (0, 2)	-496.0
  (0, 3)	-464.0
  (0, 4)	-480.0
  (0, 5)	-496.0 

quadratic dict w/ index:	 {}
quadratic dict w/ name:		 {}
symmetric quadratic dict w/ name:	 {}
quadratic matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 

symmetric quadratic matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.

In [46]:
c_vec=mod.objective.linear.to_array()
c_vec

array([-464., -480., -496., -464., -480., -496.,    0.,    0.,    0.,
          0.,    0.,    0.])

In [47]:
print(c_vec.shape)
c_vec.reshape(1,12)

(12,)


array([[-464., -480., -496., -464., -480., -496.,    0.,    0.,    0.,
           0.,    0.,    0.]])

In [48]:
c_vec2=np.kron(c_vec.reshape(1,12),c_vec.reshape(12,1))
c_vec2.shape

(12, 12)

In [53]:
mod2 = QuadraticProgram()
mod2.from_docplex(model)
mod2.minimize(quadratic=+c_vec2)


In [54]:
mod2.to_docplex().prettyprint()

// This file has been generated by DOcplex
// model name is: docplex_model1
// single vars section
dvar int x0;
dvar int x1;
dvar int x2;
dvar int x3;
dvar int x4;
dvar int x5;
dvar bool x6;
dvar bool x7;
dvar bool x8;
dvar bool x9;
dvar bool x10;
dvar bool x11;

minimize
 [ 215296 x0^2 + 445440 x0*x1 + 460288 x0*x2 + 430592 x0*x3 + 445440 x0*x4
 + 460288 x0*x5 + 230400 x1^2 + 476160 x1*x2 + 445440 x1*x3 + 460800 x1*x4
 + 476160 x1*x5 + 246016 x2^2 + 460288 x2*x3 + 476160 x2*x4 + 492032 x2*x5
 + 215296 x3^2 + 445440 x3*x4 + 460288 x3*x5 + 230400 x4^2 + 476160 x4*x5
 + 246016 x5^2 ];
 
subject to {
 _L_EXPR_1:
  x0 - 40 x6 <= 0;
 _L_EXPR_2:
  x1 - 30 x7 <= 0;
 _L_EXPR_3:
  x2 - 30 x8 <= 0;
 _L_EXPR_4:
  x3 - 100 x9 <= 0;
 _L_EXPR_5:
  x4 - 60 x10 <= 0;
 _L_EXPR_6:
  x5 - 45 x11 <= 0;
 _L_EXPR_7:
  x0 >= 0;
 _L_EXPR_8:
  x1 >= 0;
 _L_EXPR_9:
  x2 >= 0;
 _L_EXPR_10:
  x3 >= 0;
 _L_EXPR_11:
  x4 >= 0;
 _L_EXPR_12:
  x5 >= 0;
 _L_EXPR_13:
  x0 <= 40;
 _L_EXPR_14:
  x1 <= 30;
 _L_EXPR_15:
  

In [55]:
from qiskit.optimization.converters import QuadraticProgramToQubo
conv = QuadraticProgramToQubo()
mod_Qubo = conv.convert(mod2)


In [ ]:
%time result = exact.solve(mod_Qubo)
print(result)